In [2]:
import numpy as np
import pandas as pd
import os
from importlib import reload
from rgf.sklearn import RGFClassifier, FastRGFClassifier
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.model_selection import KFold, cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
root_dir = "/Users/zhilinyang/Desktop/work/2015"
model_name = 'ST4000DM000'
# model_name = 'ST31500541AS'
#model_name = 'Hitachi HDS722020ALA330'
# model_name = 'Hitachi HDS5C3030ALA630'

/Users/zhilinyang/anaconda3/lib/python3.6/site-packages/rgf/utils.py:199: UserWarning: Cannot find FastRGF executable files. FastRGF estimators will be unavailable for usage.
  warnings.warn("Cannot find FastRGF executable files. FastRGF estimators will be unavailable for usage.")


In [4]:
test = pd.read_csv('/Users/zhilinyang/Desktop/work/2015/2015-01-01.csv')
test = test.loc[test['model'] == model_name].dropna(axis=1, how='all')
test.head()
# take out one model's samples and drop empty columns 

,date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_3_normalized,smart_3_raw,smart_4_normalized,...,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_240_normalized,smart_240_raw,smart_241_normalized,smart_241_raw,smart_242_normalized,smart_242_raw
5,2015-01-01,Z3023VGH,ST4000DM000,4000787030016,0,119,213275520,99,0,100,...,100,0,200,0,100.0,1.786000e+03,100.0,8.519286e+09,100.0,3.753452e+10
7,2015-01-01,Z300JA8E,ST4000DM000,4000787030016,0,119,208002360,93,0,100,...,100,0,200,0,100.0,2.692730e+14,100.0,2.123515e+10,100.0,1.007459e+11
32,2015-01-01,Z300H2KS,ST4000DM000,4000787030016,0,117,162117824,92,0,100,...,100,0,200,0,100.0,1.462436e+13,100.0,1.521959e+10,100.0,4.747101e+10
33,2015-01-01,Z300X922,ST4000DM000,4000787030016,0,117,138875976,96,0,100,...,100,0,200,0,100.0,2.209718e+14,100.0,1.690772e+10,100.0,1.532546e+11
35,2015-01-01,Z30111FC,ST4000DM000,4000787030016,0,115,99626592,97,0,100,...,100,0,200,0,100.0,2.800877e+14,100.0,1.531666e+10,100.0,1.338117e+11


In [3]:
features_list = test.columns[5:]
print(features_list)

NameError: name 'test' is not defined

In [23]:
#combine all dates' data into a single file
def file_preprocessing(model_name):
    total_df = pd.DataFrame()
    type_dict = {feature: np.float32 for feature in features_list}
    type_dict['failure']=np.int8
    for file in os.listdir(root_dir):
        if file=='.DS_Store':
            continue
        file_path = root_dir + '/' + file
        df = pd.read_csv(file_path, dtype=type_dict)
        df = df.loc[df['model']== model_name].dropna(axis=1,how='all')
        total_df = total_df.append(df)
        print(file)
    total_df=total_df.drop(['model','capacity_bytes'],axis=1)
    return total_df

In [24]:
total_df=file_preprocessing(model_name)

2015-01-01.csv
2015-01-02.csv
2015-01-03.csv
2015-01-04.csv
2015-01-05.csv
2015-01-06.csv
2015-01-07.csv
2015-01-08.csv
2015-01-09.csv
2015-01-10.csv
2015-01-11.csv
2015-01-12.csv
2015-01-13.csv
2015-01-14.csv
2015-01-15.csv
2015-01-16.csv
2015-01-17.csv
2015-01-18.csv
2015-01-19.csv
2015-01-20.csv
2015-01-21.csv
2015-01-22.csv
2015-01-23.csv
2015-01-24.csv
2015-01-25.csv
2015-01-26.csv
2015-01-27.csv
2015-01-28.csv
2015-01-29.csv
2015-01-30.csv
2015-01-31.csv
2015-02-01.csv
2015-02-02.csv
2015-02-03.csv
2015-02-04.csv
2015-02-05.csv
2015-02-06.csv
2015-02-07.csv
2015-02-08.csv
2015-02-09.csv
2015-02-10.csv
2015-02-11.csv
2015-02-12.csv
2015-02-13.csv
2015-02-14.csv
2015-02-15.csv
2015-02-16.csv
2015-02-17.csv
2015-02-18.csv
2015-02-19.csv
2015-02-20.csv
2015-02-21.csv
2015-02-22.csv
2015-02-23.csv
2015-02-24.csv
2015-02-25.csv
2015-02-26.csv
2015-02-27.csv
2015-02-28.csv
2015-03-01.csv
2015-03-02.csv
2015-03-03.csv
2015-03-04.csv
2015-03-05.csv
2015-03-06.csv
2015-03-07.csv
2015-03-08

In [25]:
total_df.head()

,date,serial_number,failure,smart_1_normalized,smart_1_raw,smart_3_normalized,smart_3_raw,smart_4_normalized,smart_4_raw,smart_5_normalized,...,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_240_normalized,smart_240_raw,smart_241_normalized,smart_241_raw,smart_242_normalized,smart_242_raw
5,2015-01-01,Z3023VGH,0,119.0,213275520.0,99.0,0.0,100.0,1.0,100.0,...,100.0,0.0,200.0,0.0,100.0,1.786000e+03,100.0,8.519286e+09,100.0,3.753452e+10
7,2015-01-01,Z300JA8E,0,119.0,208002368.0,93.0,0.0,100.0,7.0,100.0,...,100.0,0.0,200.0,0.0,100.0,2.692730e+14,100.0,2.123515e+10,100.0,1.007459e+11
32,2015-01-01,Z300H2KS,0,117.0,162117824.0,92.0,0.0,100.0,8.0,100.0,...,100.0,0.0,200.0,0.0,100.0,1.462436e+13,100.0,1.521959e+10,100.0,4.747101e+10
33,2015-01-01,Z300X922,0,117.0,138875968.0,96.0,0.0,100.0,4.0,100.0,...,100.0,0.0,200.0,0.0,100.0,2.209718e+14,100.0,1.690772e+10,100.0,1.532546e+11
35,2015-01-01,Z30111FC,0,115.0,99626592.0,97.0,0.0,100.0,3.0,100.0,...,100.0,0.0,200.0,0.0,100.0,2.800877e+14,100.0,1.531666e+10,100.0,1.338117e+11


In [26]:
total_df.to_csv(path_or_buf=r"/Users/zhilinyang/Desktop/research/STA_combine.csv")

In [4]:
total_df=pd.read_csv(r"/Users/zhilinyang/Desktop/work/STA_combine.csv")

In [5]:
fail_names = total_df.loc[total_df['failure'] == 1]['serial_number'].unique()
print(model_name)
print('n_disks: ' + str(total_df['serial_number'].unique().size))
print('n_failed: ' + str(fail_names.size))

ST4000DM000
n_disks: 29670
n_failed: 586


In [29]:
# failure_disks: select all failed disks with all its records from total_df
failure_disk= total_df.loc[total_df['serial_number'].isin(fail_names)]
failure_disk_group=failure_disk.sort_values('date', ascending=True).groupby('serial_number')

In [30]:
#Modified version of cpt
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import bernoulli, norm, poisson
import sys

matplotlib.rc('font', size=20)
matplotlib.rc('font', family='Arial')

class cpt:
    def __init__(self, data = [], type = 'normal-mean'):
        # data (np array) as input 
        self.data = data
        self.labels = {"xlabel":"Days","ylabel":"Visits"}
        self.type = type

    def plot_data(self,type="ts",p=None):
        fig = plt.figure(figsize=(10,6))
        n = len(self.data)

        marker = ''
        linestyle = '-'

        plt.plot(np.arange(1,n+1),self.data,ls=linestyle,marker=marker)
        plt.xlabel(self.labels["xlabel"])
        plt.ylabel(self.labels["ylabel"])
        plt.ylim([0.9*np.min(self.data),1.1*np.max(self.data)])
        fig.set_tight_layout(True)
        if type=="cpt":
            tau = p[0]
            m1 = p[1]
            m2 = p[2]
            plt.plot([0,tau-1],[m1,m1],'r',lw=2)
            plt.plot([tau,n],[m2,m2],'r',lw=2)
            plt.plot([tau,tau],[0.9*np.min(self.data),1.1*np.max(self.data)],'r--',lw=2)
            filename = self.type + "-cpt.png"
            plt.savefig(filename,format="png")
        filename = self.type + ".png"
        plt.savefig(filename,format="png")
        plt.show()

    def plot_score(self):
        fig = plt.figure(figsize=(10,6))
        plt.plot(self.score)
        plt.xlabel(self.labels["xlabel"])
        plt.ylabel("Score")
        fig.set_tight_layout(True)
        filename = self.type + "-score.png"
        plt.ylim([0.,1.1*np.max(self.score)])
        plt.savefig(filename,format="png")
        plt.show()

    def find_changepoint(self):
        data = self.data
        n = len(data)
        tau = np.arange(1,n)
        lmbd = 2*np.log(n) #Bayesian Information Criterion
        eps = 1.e-8 #to avoid zeros in denominator
        if self.type=="normal-mean":
            mu0 = np.mean(data)
            s0 = np.sum((data-mu0)**2)
            s1 = np.asarray([np.sum((data[0:i]-np.mean(data[0:i]))**2) for i in range(1,n)])
            s2 = np.asarray([np.sum((data[i:]-np.mean(data[i:]))**2) for i in range(1,n)])
            R  = s0-s1-s2
            try: 
                G  = np.max(R)
            except ValueError:  #raised if `R` is empty.
                return -1
            taustar = int(np.where(R==G)[0][0]) + 1
            sd1 = np.std(data[0:taustar-1])
            sd2 = np.std(data[taustar-1:])
            #use pooled standard deviation
            var = ( taustar*sd1**2 + (n-taustar)*sd2**2 ) / n
            result = self.test_decision(2*G,var*lmbd,data,taustar)
        elif self.type=="normal-var":
            std0 = np.std(data)
            std1 = np.asarray([np.std(data[0:i]) for i in range(1,n)],dtype=float) + eps
            std2 = np.asarray([np.std(data[i:]) for i in range(1,n)],dtype=float) + eps
            R = n*np.log(std0) - tau*np.log(std1) - (n-tau)*np.log(std2)
            G  = np.max(R)
            taustar = int(np.where(R==G)[0]) + 1
            result = self.test_decision(2*G,lmbd,data,taustar)
        self.score = R
        return result

    def test_decision(self,teststat,criterion,data,tau):
        # print("---------------------")
        # print("2G = %e"%(teststat))
        # print("sigma**2*lambda = %e"%(criterion))
        if teststat > criterion:
            # print("-->H0 rejected")
            # print("Changepoint detected at position: %d"%tau)
            # m1 = np.mean(data[0:tau])
            # std1 = np.std(data[0:tau])
            # m2 = np.mean(data[tau:])
            # std2 = np.std(data[tau:])
            # if "mean" in self.type:
            #     print("m1 = %f"%m1)
            #     print("m2 = %f"%m2)
            # else:
            #     print("std1 = %f"%std1)
            #     print("std2 = %f"%std2)
            return tau
            # self.plot_data(type="cpt",p=[tau,m1,m2])
        else:
            return -1
        #     print("-->H0 not rejected")
        # print("---------------------")

if __name__ == "__main__":
    print(cpt(np.ones((60,)), 'normal-mean').find_changepoint())
    

-1


/Users/zhilinyang/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/Users/zhilinyang/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/Users/zhilinyang/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [37]:
test = failure_disk_group.get_group(fail_names[20])['smart_242_raw'].values
tmp = cpt(data=pd.to_numeric(test), type='normal-mean')
tmp.find_changepoint()

6

In [39]:
def get_cpt(data):
    changepoint = cpt(data=data, type='normal-mean').find_changepoint()
    if changepoint > 0:
        return data.size - changepoint
    return changepoint
functions_group = {n: get_cpt for n in features_list}
all_cpt_series = failure_disk_group.agg(functions_group)
print(all_cpt_series)
#find all changepoint for failed disks

               smart_1_normalized  smart_1_raw  smart_3_normalized  \
serial_number                                                        
S3001XAT                      5.0          5.0                -1.0   
S3006NW1                     41.0         63.0                -1.0   
S3006XVN                     -1.0         -1.0                -1.0   
S30081G7                     -1.0         -1.0                -1.0   
S300AS9Y                     -1.0         -1.0                -1.0   
S300HS96                     58.0         12.0               132.0   
S300VKRR                     -1.0          6.0                -1.0   
S300WD08                      7.0          8.0                 7.0   
S300WD3Q                     47.0         47.0                53.0   
S300WDD5                     -1.0         -1.0                -1.0   
S300WFMY                     86.0         86.0                -1.0   
S300X3ZE                     29.0         20.0                -1.0   
S300X8SC            

In [40]:
all_cpt_series.to_csv('/Users/zhilinyang/Desktop/all_cpt_series.csv')
print('done')

done


In [41]:
def get_percent(data):
    return data[(data>0) & (data <=100)].dropna().size/data.dropna().size
def get_median(data):
    return data[data>0].dropna().median()
def get_mean(data):
    return data[data>0].dropna().mean()
summarize = all_cpt_series.agg([get_percent, get_median, get_mean])
print(summarize)
# print(all_cpt_series['smart_194_normalized'])
# all_cpt_series


             smart_1_normalized  smart_1_raw  smart_3_normalized  smart_3_raw  \
get_percent            0.317406     0.298635            0.240614          0.0   
get_median            43.000000    50.500000           73.000000          NaN   
get_mean              74.015327    83.786263           86.931503          NaN   

             smart_4_normalized  smart_4_raw  smart_5_normalized  smart_5_raw  \
get_percent            0.003413     0.346416            0.081911      0.18430   
get_median             1.000000    75.000000            5.000000      3.00000   
get_mean               1.000000    88.841118           25.705883     27.89916   

             smart_7_normalized  smart_7_raw      ...        \
get_percent            0.488055     0.631399      ...         
get_median            94.000000    74.500000      ...         
get_mean             110.237915    88.574654      ...         

             smart_198_normalized  smart_198_raw  smart_199_normalized  \
get_percent            

In [42]:
summarize.to_csv('/Users/zhilinyang/Desktop/work/summarize.csv')
print('done')

done


In [6]:
summarize=pd.read_csv('/Users/zhilinyang/Desktop/work/summarize.csv')
summarize.head()

,Unnamed: 0,smart_1_normalized,smart_1_raw,smart_3_normalized,smart_3_raw,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,...,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_240_normalized,smart_240_raw,smart_241_normalized,smart_241_raw,smart_242_normalized,smart_242_raw
0,get_percent,0.317406,0.298635,0.240614,0.0,0.003413,0.346416,0.081911,0.18430,0.488055,...,0.064846,0.460751,0.001706,0.018771,0.0,0.445392,0.0,0.532423,0.0,0.583618
1,get_median,43.000000,50.500000,73.000000,NaN,1.000000,75.000000,5.000000,3.00000,94.000000,...,1.000000,6.000000,3.000000,22.000000,NaN,60.000000,NaN,88.000000,NaN,79.000000
2,get_mean,74.015327,83.786263,86.931503,NaN,1.000000,88.841118,25.705883,27.89916,110.237915,...,6.538462,20.100359,3.000000,29.636364,NaN,71.458336,NaN,96.736382,NaN,90.750435


In [7]:
#feature selection
row=summarize.iloc[:,0]
summarize=summarize.iloc[:,1:]
summarize.index=row
summarize.head()

,smart_1_normalized,smart_1_raw,smart_3_normalized,smart_3_raw,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,...,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_240_normalized,smart_240_raw,smart_241_normalized,smart_241_raw,smart_242_normalized,smart_242_raw
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
get_percent,0.317406,0.298635,0.240614,0.0,0.003413,0.346416,0.081911,0.18430,0.488055,0.631399,...,0.064846,0.460751,0.001706,0.018771,0.0,0.445392,0.0,0.532423,0.0,0.583618
get_median,43.000000,50.500000,73.000000,NaN,1.000000,75.000000,5.000000,3.00000,94.000000,74.500000,...,1.000000,6.000000,3.000000,22.000000,NaN,60.000000,NaN,88.000000,NaN,79.000000
get_mean,74.015327,83.786263,86.931503,NaN,1.000000,88.841118,25.705883,27.89916,110.237915,88.574654,...,6.538462,20.100359,3.000000,29.636364,NaN,71.458336,NaN,96.736382,NaN,90.750435


In [4]:
stat=(summarize.iloc[0:1]).T


In [6]:
stat.to_csv('/Users/zhilinyang/Desktop/work/stat.csv')

In [7]:
new_stat=stat[stat>=0.1]
new_stat=new_stat.dropna()
features=list(new_stat.index)
features

['smart_1_normalized',
 'smart_1_raw',
 'smart_3_normalized',
 'smart_4_raw',
 'smart_5_raw',
 'smart_7_normalized',
 'smart_7_raw',
 'smart_9_normalized',
 'smart_9_raw',
 'smart_12_raw',
 'smart_187_normalized',
 'smart_187_raw',
 'smart_189_normalized',
 'smart_189_raw',
 'smart_190_normalized',
 'smart_190_raw',
 'smart_193_normalized',
 'smart_193_raw',
 'smart_194_normalized',
 'smart_194_raw',
 'smart_197_raw',
 'smart_198_raw',
 'smart_240_raw',
 'smart_241_raw',
 'smart_242_raw']

In [8]:
Sgt_features = ['smart_1_normalized', 'smart_1_raw', 'smart_5_normalized', 'smart_5_raw', 'smart_7_normalized', 'smart_7_raw',
    'smart_184_normalized', 'smart_184_raw', 'smart_187_normalized', 'smart_187_raw', 'smart_188_raw', 'smart_189_normalized', 'smart_189_raw', 
    'smart_190_normalized', 'smart_190_raw', 'smart_193_normalized', 'smart_193_raw', 'smart_194_normalized', 'smart_194_raw', 'smart_197_normalized', 
    'smart_197_raw', 'smart_198_normalized', 'smart_198_raw', 'smart_240_raw', 'smart_241_raw', 'smart_242_raw', 'failure']
new_features = [i for i in features if i not in Sgt_features]
print(new_features)

['smart_3_normalized', 'smart_4_raw', 'smart_9_normalized', 'smart_9_raw', 'smart_12_raw']


In [6]:
total_df=pd.read_csv(r"/Users/zhilinyang/Desktop/work/STA_combine.csv")
total_df=total_df.iloc[:,1:]
total_df.head()

,date,serial_number,failure,smart_1_normalized,smart_1_raw,smart_3_normalized,smart_3_raw,smart_4_normalized,smart_4_raw,smart_5_normalized,...,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_240_normalized,smart_240_raw,smart_241_normalized,smart_241_raw,smart_242_normalized,smart_242_raw
0,2015-01-01,Z3023VGH,0,119.0,213275520.0,99.0,0.0,100.0,1.0,100.0,...,100.0,0.0,200.0,0.0,100.0,1.786000e+03,100.0,8.519286e+09,100.0,3.753452e+10
1,2015-01-01,Z300JA8E,0,119.0,208002368.0,93.0,0.0,100.0,7.0,100.0,...,100.0,0.0,200.0,0.0,100.0,2.692730e+14,100.0,2.123515e+10,100.0,1.007459e+11
2,2015-01-01,Z300H2KS,0,117.0,162117824.0,92.0,0.0,100.0,8.0,100.0,...,100.0,0.0,200.0,0.0,100.0,1.462436e+13,100.0,1.521959e+10,100.0,4.747101e+10
3,2015-01-01,Z300X922,0,117.0,138875968.0,96.0,0.0,100.0,4.0,100.0,...,100.0,0.0,200.0,0.0,100.0,2.209718e+14,100.0,1.690772e+10,100.0,1.532546e+11
4,2015-01-01,Z30111FC,0,115.0,99626592.0,97.0,0.0,100.0,3.0,100.0,...,100.0,0.0,200.0,0.0,100.0,2.800877e+14,100.0,1.531666e+10,100.0,1.338117e+11


In [6]:
disk_list=total_df['serial_number'].unique()
len(disk_list)

29670

In [8]:
import sys
num=0
features=['smart_1_normalized',
 'smart_1_raw',
 'smart_3_normalized',
 'smart_4_raw',
 'smart_5_raw',
 'smart_7_normalized',
 'smart_7_raw',
 'smart_9_normalized',
 'smart_9_raw',
 'smart_12_raw',
 'smart_187_normalized',
 'smart_187_raw',
 'smart_189_normalized',
 'smart_189_raw',
 'smart_190_normalized',
 'smart_190_raw',
 'smart_193_normalized',
 'smart_193_raw',
 'smart_194_normalized',
 'smart_194_raw',
 'smart_197_raw',
 'smart_198_raw',
 'smart_240_raw',
 'smart_241_raw',
 'smart_242_raw']
groups=total_df.groupby('serial_number')
x_train=pd.DataFrame()
for sample_name in disk_list:
    sample=groups.get_group(sample_name)
    train_sample=[]
    for feature in features:
        i=summarize.loc['get_median',feature]
        arr=(sample[feature].ewm(span=i).mean()).values
        train_sample.append(arr[-1])
    train_vals=(pd.DataFrame(train_sample)).T
    x_train=x_train.append(train_vals,ignore_index=True)
    num=num+1
    sys.stdout.write(str(num) + '\n')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074


3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716


5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346
5347
5348
5349
5350
5351
5352
5353
5354
5355
5356
5357
5358
5359
5360
5361
5362
5363
5364
5365


6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978
6979
6980
6981
6982
6983
6984
6985
6986
6987
6988
6989
6990
6991
6992
6993
6994
6995
6996
6997
6998
6999
7000
7001
7002
7003
7004
7005
7006
7007
7008


...

23813
23814
23815
23816
23817
23818
23819
23820
23821
23822
23823
23824
23825
23826
23827
23828
23829
23830
23831
23832
23833
23834
23835
23836
23837
23838
23839
23840
23841
23842
23843
23844
23845
23846
23847
23848
23849
23850
23851
23852
23853
23854
23855
23856
23857
23858
23859
23860
23861
23862
23863
23864
23865
23866
23867
23868
23869
23870
23871
23872
23873
23874
23875
23876
23877
23878
23879
23880
23881
23882
23883
23884
23885
23886
23887
23888
23889
23890
23891
23892
23893
23894
23895
23896
23897
23898
23899
23900
23901
23902
23903
23904
23905
23906
23907
23908
23909
23910
23911
23912
23913
23914
23915
23916
23917
23918
23919
23920
23921
23922
23923
23924
23925
23926
23927
23928
23929
23930
23931
23932
23933
23934
23935
23936
23937
23938
23939
23940
23941
23942
23943
23944
23945
23946
23947
23948
23949
23950
23951
23952
23953
23954
23955
23956
23957
23958
23959
23960
23961
23962
23963
23964
23965
23966
23967
23968
23969
23970
23971
23972
23973
23974
23975
23976
23977
23978
2397

25183
25184
25185
25186
25187
25188
25189
25190
25191
25192
25193
25194
25195
25196
25197
25198
25199
25200
25201
25202
25203
25204
25205
25206
25207
25208
25209
25210
25211
25212
25213
25214
25215
25216
25217
25218
25219
25220
25221
25222
25223
25224
25225
25226
25227
25228
25229
25230
25231
25232
25233
25234
25235
25236
25237
25238
25239
25240
25241
25242
25243
25244
25245
25246
25247
25248
25249
25250
25251
25252
25253
25254
25255
25256
25257
25258
25259
25260
25261
25262
25263
25264
25265
25266
25267
25268
25269
25270
25271
25272
25273
25274
25275
25276
25277
25278
25279
25280
25281
25282
25283
25284
25285
25286
25287
25288
25289
25290
25291
25292
25293
25294
25295
25296
25297
25298
25299
25300
25301
25302
25303
25304
25305
25306
25307
25308
25309
25310
25311
25312
25313
25314
25315
25316
25317
25318
25319
25320
25321
25322
25323
25324
25325
25326
25327
25328
25329
25330
25331
25332
25333
25334
25335
25336
25337
25338
25339
25340
25341
25342
25343
25344
25345
25346
25347
25348
2534

26558
26559
26560
26561
26562
26563
26564
26565
26566
26567
26568
26569
26570
26571
26572
26573
26574
26575
26576
26577
26578
26579
26580
26581
26582
26583
26584
26585
26586
26587
26588
26589
26590
26591
26592
26593
26594
26595
26596
26597
26598
26599
26600
26601
26602
26603
26604
26605
26606
26607
26608
26609
26610
26611
26612
26613
26614
26615
26616
26617
26618
26619
26620
26621
26622
26623
26624
26625
26626
26627
26628
26629
26630
26631
26632
26633
26634
26635
26636
26637
26638
26639
26640
26641
26642
26643
26644
26645
26646
26647
26648
26649
26650
26651
26652
26653
26654
26655
26656
26657
26658
26659
26660
26661
26662
26663
26664
26665
26666
26667
26668
26669
26670
26671
26672
26673
26674
26675
26676
26677
26678
26679
26680
26681
26682
26683
26684
26685
26686
26687
26688
26689
26690
26691
26692
26693
26694
26695
26696
26697
26698
26699
26700
26701
26702
26703
26704
26705
26706
26707
26708
26709
26710
26711
26712
26713
26714
26715
26716
26717
26718
26719
26720
26721
26722
26723
2672

27930
27931
27932
27933
27934
27935
27936
27937
27938
27939
27940
27941
27942
27943
27944
27945
27946
27947
27948
27949
27950
27951
27952
27953
27954
27955
27956
27957
27958
27959
27960
27961
27962
27963
27964
27965
27966
27967
27968
27969
27970
27971
27972
27973
27974
27975
27976
27977
27978
27979
27980
27981
27982
27983
27984
27985
27986
27987
27988
27989
27990
27991
27992
27993
27994
27995
27996
27997
27998
27999
28000
28001
28002
28003
28004
28005
28006
28007
28008
28009
28010
28011
28012
28013
28014
28015
28016
28017
28018
28019
28020
28021
28022
28023
28024
28025
28026
28027
28028
28029
28030
28031
28032
28033
28034
28035
28036
28037
28038
28039
28040
28041
28042
28043
28044
28045
28046
28047
28048
28049
28050
28051
28052
28053
28054
28055
28056
28057
28058
28059
28060
28061
28062
28063
28064
28065
28066
28067
28068
28069
28070
28071
28072
28073
28074
28075
28076
28077
28078
28079
28080
28081
28082
28083
28084
28085
28086
28087
28088
28089
28090
28091
28092
28093
28094
28095
2809

29309
29310
29311
29312
29313
29314
29315
29316
29317
29318
29319
29320
29321
29322
29323
29324
29325
29326
29327
29328
29329
29330
29331
29332
29333
29334
29335
29336
29337
29338
29339
29340
29341
29342
29343
29344
29345
29346
29347
29348
29349
29350
29351
29352
29353
29354
29355
29356
29357
29358
29359
29360
29361
29362
29363
29364
29365
29366
29367
29368
29369
29370
29371
29372
29373
29374
29375
29376
29377
29378
29379
29380
29381
29382
29383
29384
29385
29386
29387
29388
29389
29390
29391
29392
29393
29394
29395
29396
29397
29398
29399
29400
29401
29402
29403
29404
29405
29406
29407
29408
29409
29410
29411
29412
29413
29414
29415
29416
29417
29418
29419
29420
29421
29422
29423
29424
29425
29426
29427
29428
29429
29430
29431
29432
29433
29434
29435
29436
29437
29438
29439
29440
29441
29442
29443
29444
29445
29446
29447
29448
29449
29450
29451
29452
29453
29454
29455
29456
29457
29458
29459
29460
29461
29462
29463
29464
29465
29466
29467
29468
29469
29470
29471
29472
29473
29474
2947

In [12]:
x_train.loc['Z3023VGH']

smart_1_normalized      1.149571e+02
smart_1_raw             1.177851e+08
smart_3_normalized      9.305554e+01
smart_4_raw             6.939883e+00
smart_5_raw             0.000000e+00
smart_7_normalized      8.641512e+01
smart_7_raw             4.764433e+09
smart_9_normalized      8.955260e+01
smart_9_raw             9.710866e+03
smart_12_raw            6.939883e+00
smart_187_normalized    1.000000e+02
smart_187_raw           0.000000e+00
smart_189_normalized    1.000000e+02
smart_189_raw           0.000000e+00
smart_190_normalized    7.438765e+01
smart_190_raw           2.561235e+01
smart_193_normalized    8.405032e+01
smart_193_raw           3.237897e+04
smart_194_normalized    2.561235e+01
smart_194_raw           2.561235e+01
smart_197_raw           0.000000e+00
smart_198_raw           0.000000e+00
smart_240_raw           9.766872e+03
smart_241_raw           1.607025e+10
smart_242_raw           1.187183e+11
Name: Z3023VGH, dtype: float64

In [10]:
x_train.to_csv('/Users/zhilinyang/Desktop/work/SGTA_x_train.csv')

In [7]:
x_train=pd.read_csv('/Users/zhilinyang/Desktop/work/SGTA_x_train.csv')
x_train.head()

,Unnamed: 0,smart_1_normalized,smart_1_raw,smart_3_normalized,smart_4_raw,smart_5_raw,smart_7_normalized,smart_7_raw,smart_9_normalized,smart_9_raw,...,smart_190_raw,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_197_raw,smart_198_raw,smart_240_raw,smart_241_raw,smart_242_raw
0,Z3023VGH,114.957068,1.177851e+08,93.055541,6.939883,0.0,86.415124,4.764433e+09,89.552597,9710.866056,...,25.612355,84.050316,32378.966059,25.612355,25.612355,0.0,0.0,9.766872e+03,1.607025e+10,1.187183e+11
1,Z300JA8E,115.546442,1.241334e+08,93.000000,7.000000,0.0,86.881700,5.639896e+08,78.596499,19312.647053,...,22.685215,100.000000,664.871270,22.685215,22.685215,0.0,0.0,1.270232e+14,2.605104e+10,1.221685e+11
2,Z300H2KS,115.537885,1.310872e+08,92.000000,10.886164,0.0,79.997877,3.525747e+08,81.790694,16541.502638,...,23.036176,85.124305,31697.342098,23.036176,23.036176,0.0,0.0,1.185004e+14,1.867884e+10,5.862368e+10
3,Z300X922,114.787406,1.274106e+08,93.338804,6.641985,0.0,84.686820,3.606703e+08,81.198093,16979.758324,...,22.980248,96.360385,8070.795361,22.980248,22.980248,0.0,0.0,1.327738e+14,2.242619e+10,2.188136e+11
4,Z30111FC,114.301742,1.141041e+08,96.268400,3.722028,0.0,83.050088,2.427153e+08,82.694100,15758.683878,...,19.604739,99.000000,3265.517553,19.604739,19.604739,0.0,0.0,3.067075e+13,2.097119e+10,1.928540e+11


In [8]:
x_train['serial_number']=x_train.iloc[:,0]
x_train=x_train.iloc[:,1:]
x_train.head()

,smart_1_normalized,smart_1_raw,smart_3_normalized,smart_4_raw,smart_5_raw,smart_7_normalized,smart_7_raw,smart_9_normalized,smart_9_raw,smart_12_raw,...,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_197_raw,smart_198_raw,smart_240_raw,smart_241_raw,smart_242_raw,serial_number
0,114.957068,1.177851e+08,93.055541,6.939883,0.0,86.415124,4.764433e+09,89.552597,9710.866056,6.939883,...,84.050316,32378.966059,25.612355,25.612355,0.0,0.0,9.766872e+03,1.607025e+10,1.187183e+11,Z3023VGH
1,115.546442,1.241334e+08,93.000000,7.000000,0.0,86.881700,5.639896e+08,78.596499,19312.647053,7.000000,...,100.000000,664.871270,22.685215,22.685215,0.0,0.0,1.270232e+14,2.605104e+10,1.221685e+11,Z300JA8E
2,115.537885,1.310872e+08,92.000000,10.886164,0.0,79.997877,3.525747e+08,81.790694,16541.502638,9.886164,...,85.124305,31697.342098,23.036176,23.036176,0.0,0.0,1.185004e+14,1.867884e+10,5.862368e+10,Z300H2KS
3,114.787406,1.274106e+08,93.338804,6.641985,0.0,84.686820,3.606703e+08,81.198093,16979.758324,6.641985,...,96.360385,8070.795361,22.980248,22.980248,0.0,0.0,1.327738e+14,2.242619e+10,2.188136e+11,Z300X922
4,114.301742,1.141041e+08,96.268400,3.722028,0.0,83.050088,2.427153e+08,82.694100,15758.683878,3.722028,...,99.000000,3265.517553,19.604739,19.604739,0.0,0.0,3.067075e+13,2.097119e+10,1.928540e+11,Z30111FC


In [9]:
fail_names = total_df.loc[total_df['failure'] == 1]['serial_number'].unique()
x_train['failure'] = x_train.apply(lambda row: 1 if row['serial_number'] in fail_names else 0, axis=1)

In [12]:
# kmeans
# prepare set
x_train = x_train.dropna(how='any')
n_clusters = fail_names.size // 5
n_clusters

117

In [19]:
X_health = x_train.loc[x_train['failure'] == 0].drop(['serial_number','failure'], axis=1).values
kmeans = KMeans(n_clusters=n_clusters, random_state=0, n_jobs=-1).fit(X_health)

In [20]:
X_health_transformed = []
for j in range(0, n_clusters):
    d = kmeans.transform(X_health)[:, j]
    ind = np.argsort(d)[:10]
    X_health_transformed[0:0] = list(X_health[ind])

In [27]:
X_health_transformed=pd.DataFrame(X_health_transformed)

In [10]:
X_health_transformed=pd.read_csv("/Users/zhilinyang/Desktop/work/x_health.csv")

In [11]:
X_health_transformed=X_health_transformed.iloc[:,1:]
X_health_transformed.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,114.770837,1.116605e+08,95.136215,4.855688,0.0,82.592930,2.143100e+08,83.452913,15036.980138,4.855688,...,27.116284,100.000000,1562.830105,27.116284,27.116284,0.0,0.0,1.293401e+13,2.219247e+10,1.181882e+11
1,114.821303,1.038086e+08,92.000000,21.887076,0.0,84.380478,3.319070e+08,81.147965,17019.000307,21.887076,...,21.220992,87.346070,26072.198547,21.220992,21.220992,0.0,0.0,1.297154e+13,2.433229e+10,1.344223e+11
2,114.472452,1.057519e+08,91.000000,17.699002,0.0,83.492508,2.585328e+08,78.095012,19699.304579,16.699002,...,25.779816,79.131602,42312.760938,25.779816,25.779816,0.0,0.0,1.293210e+13,2.557531e+10,1.051124e+11
3,116.170930,1.321802e+08,92.000000,23.699002,0.0,83.566904,2.663788e+08,78.020246,19769.194192,23.699002,...,26.696146,77.127897,46868.450343,26.696146,26.696146,0.0,0.0,1.296764e+13,2.553865e+10,8.995254e+10
4,115.587858,1.310163e+08,91.000053,8.864692,0.0,82.582077,5.068196e+08,83.446646,15040.451193,8.864692,...,30.139005,92.000026,17696.521262,30.139005,30.139005,0.0,0.0,1.292174e+13,2.000466e+10,1.559076e+11


In [12]:
X_health_transformed=list(np.array(X_health_transformed))
X_health_transformed

[array([  1.14770837e+02,   1.11660461e+08,   9.51362152e+01,
          4.85568831e+00,   0.00000000e+00,   8.25929295e+01,
          2.14310011e+08,   8.34529125e+01,   1.50369801e+04,
          4.85568831e+00,   1.00000000e+02,   0.00000000e+00,
          8.90940218e+01,   1.09072034e+01,   7.28837159e+01,
          2.71162841e+01,   1.00000000e+02,   1.56283011e+03,
          2.71162841e+01,   2.71162841e+01,   0.00000000e+00,
          0.00000000e+00,   1.29340082e+13,   2.21924656e+10,
          1.18188237e+11]),
 array([  1.14821303e+02,   1.03808605e+08,   9.20000000e+01,
          2.18870756e+01,   0.00000000e+00,   8.43804778e+01,
          3.31907046e+08,   8.11479652e+01,   1.70190003e+04,
          2.18870756e+01,   1.00000000e+02,   0.00000000e+00,
          5.06533717e+01,   4.93523272e+01,   7.87790077e+01,
          2.12209923e+01,   8.73460699e+01,   2.60721985e+04,
          2.12209923e+01,   2.12209923e+01,   0.00000000e+00,
          0.00000000e+00,   1.29715449e+13

In [13]:
X_failed = list(x_train.loc[x_train['failure'] == 1].drop(['serial_number', 'failure'], axis=1).values)
X = np.array(X_health_transformed + X_failed)
y = np.concatenate((np.zeros(len(X_health_transformed)), np.ones(len(X_failed))), axis=0) 
print(X.shape)
print(y.shape)

(1756, 25)
(1756,)


In [14]:
def model_stat(model, X, y):
    f_score = cross_val_score(model, X, y, cv=5, scoring='f1', n_jobs=-1)
    r_score = cross_val_score(model, X, y, cv=5, scoring='recall', n_jobs=-1)
    p_score = cross_val_score(model, X, y, cv=5, scoring='precision', n_jobs=-1)
    return pd.Series({'P': np.mean(p_score), 'R': np.mean(r_score), 'F': np.mean(f_score), 'Sd': np.std(f_score)})

statics = pd.DataFrame(index=['GBDT', 'SVM', 'DT', 'LR', 'RF', 'RGF'], columns=['P', 'R', 'F', 'Sd'])
statics

,P,R,F,Sd
GBDT,NaN,NaN,NaN,NaN
SVM,NaN,NaN,NaN,NaN
DT,NaN,NaN,NaN,NaN
LR,NaN,NaN,NaN,NaN
RF,NaN,NaN,NaN,NaN
RGF,NaN,NaN,NaN,NaN


In [15]:
search_grid = {
    'C': np.linspace(0.1, 0.5, 10),
    'class_weight': ['balanced'],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
    }

lr_model = LogisticRegression()
lr_grid = GridSearchCV(estimator=lr_model, param_grid=search_grid, 
    cv=5, scoring='f1', n_jobs=-1, verbose=2)

lr_grid.fit(X, y)
print(lr_grid.best_score_)
print(lr_grid.best_params_)

statics.loc['LR'] = model_stat(lr_grid.best_estimator_, X, y)
statics.loc['LR']

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] C=0.1, class_weight=balanced, solver=newton-cg ..................
[CV] C=0.1, class_weight=balanced, solver=newton-cg ..................
[CV] C=0.1, class_weight=balanced, solver=newton-cg ..................
[CV] C=0.1, class_weight=balanced, solver=newton-cg ..................
[CV] C=0.1, class_weight=balanced, solver=newton-cg ..................
[CV] C=0.1, class_weight=balanced, solver=lbfgs ......................
[CV] C=0.1, class_weight=balanced, solver=lbfgs ......................
[CV] C=0.1, class_weight=balanced, solver=lbfgs ......................
[CV] ....... C=0.1, class_weight=balanced, solver=lbfgs, total=   0.1s
[CV] C=0.1, class_weight=balanced, solver=lbfgs ......................


/Users/zhilinyang/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:462: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/zhilinyang/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/zhilinyang/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:462: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/zhilinyang/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


[CV] ....... C=0.1, class_weight=balanced, solver=lbfgs, total=   0.1s
[CV] C=0.1, class_weight=balanced, solver=lbfgs ......................
[CV] ....... C=0.1, class_weight=balanced, solver=lbfgs, total=   0.1s
[CV] C=0.1, class_weight=balanced, solver=liblinear ..................
[CV] ....... C=0.1, class_weight=balanced, solver=lbfgs, total=   0.1s
[CV] C=0.1, class_weight=balanced, solver=liblinear ..................
[CV] ... C=0.1, class_weight=balanced, solver=liblinear, total=   0.0s
[CV] C=0.1, class_weight=balanced, solver=liblinear ..................
[CV] ... C=0.1, class_weight=balanced, solver=liblinear, total=   0.0s
[CV] C=0.1, class_weight=balanced, solver=liblinear ..................
[CV] ... C=0.1, class_weight=balanced, solver=liblinear, total=   0.0s
[CV] C=0.1, class_weight=balanced, solver=liblinear ..................
[CV] ....... C=0.1, class_weight=balanced, solver=lbfgs, total=   0.1s
[CV] C=0.1, class_weight=balanced, solver=sag ........................
[CV] .

/Users/zhilinyang/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:462: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/zhilinyang/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/zhilinyang/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:462: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/zhilinyang/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/zhilinyang/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:462: LineSearchWarn

[CV] ......... C=0.1, class_weight=balanced, solver=sag, total=   0.1s
[CV] C=0.1, class_weight=balanced, solver=saga .......................


...

[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   11.6s finished
/Users/zhilinyang/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:462: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/zhilinyang/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


0.698539458153
{'C': 0.41111111111111109, 'class_weight': 'balanced', 'solver': 'newton-cg'}


/Users/zhilinyang/anaconda3/lib/python3.6/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
/Users/zhilinyang/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:422: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
/Users/zhilinyang/anaconda3/lib/python3.6/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
/Users/zhilinyang/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:422: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
/Users/zhilinyang/anaconda3/lib/python3.6/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
/Users/zhilinyang/anaconda3/lib/python3.6/site-packages/scipy/optimize/line

/Users/zhilinyang/anaconda3/lib/python3.6/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
/Users/zhilinyang/anaconda3/lib/python3.6/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


P     0.722948
R     0.687918
F     0.698563
Sd    0.087184
Name: LR, dtype: object

In [16]:
# tune RF
n_estimators = [13] # tuned
max_features = ['auto'] # tuned
criterion = ['entropy']  # tuned
max_depth = [18] # tuned
min_samples_split = [2]
min_samples_leaf = [1]
min_weight_fraction_leaf = [0]
max_leaf_nodes = [None]

bootstrap = [False]
random_state = [0]
class_weight = ['balanced']
search_grid = {
    'n_estimators': n_estimators,
    'criterion': criterion,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'min_weight_fraction_leaf': min_weight_fraction_leaf,
    'max_leaf_nodes': max_leaf_nodes,
    'bootstrap': bootstrap,
    'random_state': random_state,
    'class_weight': class_weight
    }

rf_model = RandomForestClassifier()
rf_grid = GridSearchCV(estimator=rf_model, param_grid=search_grid, 
    cv=5, scoring='f1', n_jobs=-1, verbose=2)

rf_grid.fit(X, y)
print(rf_grid.best_score_)
print(rf_grid.best_params_)

statics.loc['RF'] = model_stat(rf_grid.best_estimator_, X, y)
statics.loc['RF']

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] bootstrap=False, class_weight=balanced, criterion=entropy, max_depth=18, max_features=auto, max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0, n_estimators=13, random_state=0 
[CV] bootstrap=False, class_weight=balanced, criterion=entropy, max_depth=18, max_features=auto, max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0, n_estimators=13, random_state=0 
[CV] bootstrap=False, class_weight=balanced, criterion=entropy, max_depth=18, max_features=auto, max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0, n_estimators=13, random_state=0 
[CV] bootstrap=False, class_weight=balanced, criterion=entropy, max_depth=18, max_features=auto, max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0, n_estimators=13, random_state=0 
[CV] bootstrap=False, class_weight=balanced, criterion=entrop

[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.2s finished


0.895814351583
{'bootstrap': False, 'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 18, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0, 'n_estimators': 13, 'random_state': 0}


P      0.905384
R      0.887339
F      0.895788
Sd    0.0577791
Name: RF, dtype: object

In [32]:
# tune SVM
search_grid = {
    'C': [1.05], 
    'kernel': ['linear'],
    'gamma': [0.05],
    'class_weight': [{0:0.005,1:0.995}],
    'max_iter': [-1],
    'random_state': [0]
}
svm = SVC(C=0.001,gamma=50,class_weight={0:0.05,1:0.95})
scores=cross_val_score(estimator=svm,scoring='precision',X=X,y=y,cv=5,n_jobs=2)
print('SVM replaced precision: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))
scores=cross_val_score(estimator=svm,scoring='recall',X=X,y=y,cv=5,n_jobs=2)
print('SVM replaced recall: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))
scores=cross_val_score(estimator=svm,scoring='f1',X=X,y=y,cv=5,n_jobs=2)
print('SVM replaced f1: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))


SVM replaced precision: 0.334 +/- 0.001
SVM replaced recall: 1.000 +/- 0.000
SVM replaced f1: 0.500 +/- 0.001


In [50]:
# tune GBDT
search_grid = {
    'n_estimators': [169],
    'min_samples_split': [2],
    'min_samples_leaf': [2],
    'max_depth': [8],
    'max_features':['sqrt'],
    'subsample': [0.93],
    'random_state': [0]
}
gbdt_model = GradientBoostingClassifier()
gbdt_grid = GridSearchCV(estimator= gbdt_model, param_grid=search_grid, 
    cv=5, scoring='f1', n_jobs=-1, verbose=2)
gbdt_grid.fit(X, y)

print(gbdt_grid.best_score_)
print(gbdt_grid.best_params_)

statics.loc['GBDT'] = model_stat(gbdt_grid.best_estimator_, X, y)
statics.loc['GBDT']

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] max_depth=8, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=169, random_state=0, subsample=0.93 
[CV] max_depth=8, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=169, random_state=0, subsample=0.93 
[CV] max_depth=8, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=169, random_state=0, subsample=0.93 
[CV] max_depth=8, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=169, random_state=0, subsample=0.93 
[CV] max_depth=8, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=169, random_state=0, subsample=0.93 
[CV]  max_depth=8, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=169, random_state=0, subsample=0.93, total=   0.8s
[CV]  max_depth=8, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=169, random_state=0, subsample=0.93, total=   0.8s
[CV]  max_depth

[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.8s remaining:    1.2s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.9s finished


0.911259468075
{'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 169, 'random_state': 0, 'subsample': 0.93}


P      0.912118
R      0.911198
F      0.911226
Sd    0.0595772
Name: GBDT, dtype: object

In [14]:
X_train=X
y_train=y

In [16]:
from rgf.sklearn import RGFClassifier
RGF = RGFClassifier(max_leaf=10000,l2=1.0,sl2=0.005,test_interval=100,verbose=True)
pres=cross_val_score(estimator=RGF,scoring='precision',X=X_train,y=y_train,cv=5,n_jobs=2)
recall=cross_val_score(estimator=RGF,scoring='recall',X=X_train,y=y_train,cv=5,n_jobs=2)
f1=cross_val_score(estimator=RGF,scoring='f1',X=X_train,y=y_train,cv=5,n_jobs=2)

n_jobs = -1, but RGFClassifier uses one CPU because classes_ is 2
n_jobs = -1, but RGFClassifier uses one CPU because classes_ is 2
"train": 
   algorithm=RGF
   train_x_fn=/tmp/rgf/6a839de2-a552-40cc-8d69-53f3741b517e1.train.data.x
   train_y_fn=/tmp/rgf/6a839de2-a552-40cc-8d69-53f3741b517e1.train.data.y
   Log:ON
   model_fn_prefix=/tmp/rgf/6a839de2-a552-40cc-8d69-53f3741b517e1.model
--------------------
Mon Jul 23 15:52:24 2018: Reading training data ... 
Mon Jul 23 15:52:24 2018: Start ... #train=1404
--------------------
Forest-level: 
   loss=Log
   max_leaf_forest=10000
   max_tree=5000
   opt_interval=100
   test_interval=100
   num_tree_search=1
   Verbose:ON
   memory_policy=Generous
-------------
Training data: 25x1404, nonzero_ratio=0.8519; managed as dense data.
-------------
Optimization: 
   loss=Log
   num_iteration_opt=5
   reg_L2=1
   opt_stepsize=0.5
   max_delta=1
Tree-level: min_pop=10
Node split: reg_L2=0.005
--------------------
Mon Jul 23 15:52:24 2018: Calling 


None
"predict": 
   model_fn=/tmp/rgf/6a839de2-a552-40cc-8d69-53f3741b517e1.model-99
   test_x_fn=/tmp/rgf/6a839de2-a552-40cc-8d69-53f3741b517e1.test.data.x
   prediction_fn=/tmp/rgf/6a839de2-a552-40cc-8d69-53f3741b517e1.predictions.txt
   Log:ON
--------------------
Mon Jul 23 15:52:47 2018: Reading test data ... 
Mon Jul 23 15:52:47 2018: Predicting ... 
elapsed: 0.035116
/tmp/rgf/6a839de2-a552-40cc-8d69-53f3741b517e1.predictions.txt: /tmp/rgf/6a839de2-a552-40cc-8d69-53f3741b517e1.model-99,#leaf=9900,#tree=1791
Mon Jul 23 15:52:47 2018: Done ... 

None
n_jobs = -1, but RGFClassifier uses one CPU because classes_ is 2
"train": 
   algorithm=RGF
   train_x_fn=/tmp/rgf/33c7bd7a-b038-466a-a085-4d67903712361.train.data.x
   train_y_fn=/tmp/rgf/33c7bd7a-b038-466a-a085-4d67903712361.train.data.y
   Log:ON
   model_fn_prefix=/tmp/rgf/33c7bd7a-b038-466a-a085-4d67903712361.model
--------------------
Mon Jul 23 15:52:24 2018: Reading training data ... 
Mon Jul 23 15:52:24 2018: Start ... #trai


None
"predict": 
   model_fn=/tmp/rgf/33c7bd7a-b038-466a-a085-4d67903712361.model-99
   test_x_fn=/tmp/rgf/33c7bd7a-b038-466a-a085-4d67903712361.test.data.x
   prediction_fn=/tmp/rgf/33c7bd7a-b038-466a-a085-4d67903712361.predictions.txt
   Log:ON
--------------------
Mon Jul 23 15:52:48 2018: Reading test data ... 
Mon Jul 23 15:52:48 2018: Predicting ... 
elapsed: 0.034709
/tmp/rgf/33c7bd7a-b038-466a-a085-4d67903712361.predictions.txt: /tmp/rgf/33c7bd7a-b038-466a-a085-4d67903712361.model-99,#leaf=9900,#tree=1844
Mon Jul 23 15:52:48 2018: Done ... 

None
n_jobs = -1, but RGFClassifier uses one CPU because classes_ is 2
"train": 
   algorithm=RGF
   train_x_fn=/tmp/rgf/c890e901-d60d-4f73-8a4c-ebf4ea7a8b8a2.train.data.x
   train_y_fn=/tmp/rgf/c890e901-d60d-4f73-8a4c-ebf4ea7a8b8a2.train.data.y
   Log:ON
   model_fn_prefix=/tmp/rgf/c890e901-d60d-4f73-8a4c-ebf4ea7a8b8a2.model
--------------------
Mon Jul 23 15:52:48 2018: Reading training data ... 
Mon Jul 23 15:52:48 2018: Start ... #trai


None
"predict": 
   model_fn=/tmp/rgf/c890e901-d60d-4f73-8a4c-ebf4ea7a8b8a2.model-99
   test_x_fn=/tmp/rgf/c890e901-d60d-4f73-8a4c-ebf4ea7a8b8a2.test.data.x
   prediction_fn=/tmp/rgf/c890e901-d60d-4f73-8a4c-ebf4ea7a8b8a2.predictions.txt
   Log:ON
--------------------
Mon Jul 23 15:53:13 2018: Reading test data ... 
Mon Jul 23 15:53:13 2018: Predicting ... 
elapsed: 0.032135
/tmp/rgf/c890e901-d60d-4f73-8a4c-ebf4ea7a8b8a2.predictions.txt: /tmp/rgf/c890e901-d60d-4f73-8a4c-ebf4ea7a8b8a2.model-99,#leaf=9900,#tree=1781
Mon Jul 23 15:53:13 2018: Done ... 

None
n_jobs = -1, but RGFClassifier uses one CPU because classes_ is 2
"train": 
   algorithm=RGF
   train_x_fn=/tmp/rgf/8e4c5f17-050e-43ef-ad7a-7ac87f77bf2a2.train.data.x
   train_y_fn=/tmp/rgf/8e4c5f17-050e-43ef-ad7a-7ac87f77bf2a2.train.data.y
   Log:ON
   model_fn_prefix=/tmp/rgf/8e4c5f17-050e-43ef-ad7a-7ac87f77bf2a2.model
--------------------
Mon Jul 23 15:52:48 2018: Reading training data ... 
Mon Jul 23 15:52:48 2018: Start ... #trai


None
"predict": 
   model_fn=/tmp/rgf/8e4c5f17-050e-43ef-ad7a-7ac87f77bf2a2.model-99
   test_x_fn=/tmp/rgf/8e4c5f17-050e-43ef-ad7a-7ac87f77bf2a2.test.data.x
   prediction_fn=/tmp/rgf/8e4c5f17-050e-43ef-ad7a-7ac87f77bf2a2.predictions.txt
   Log:ON
--------------------
Mon Jul 23 15:53:13 2018: Reading test data ... 
Mon Jul 23 15:53:13 2018: Predicting ... 
elapsed: 0.033643
/tmp/rgf/8e4c5f17-050e-43ef-ad7a-7ac87f77bf2a2.predictions.txt: /tmp/rgf/8e4c5f17-050e-43ef-ad7a-7ac87f77bf2a2.model-99,#leaf=9900,#tree=1753
Mon Jul 23 15:53:13 2018: Done ... 

None
"train": 
   algorithm=RGF
   train_x_fn=/tmp/rgf/fc815fa8-cea9-409d-87bb-7d32185a3db43.train.data.x
   train_y_fn=/tmp/rgf/fc815fa8-cea9-409d-87bb-7d32185a3db43.train.data.y
   Log:ON
   model_fn_prefix=/tmp/rgf/fc815fa8-cea9-409d-87bb-7d32185a3db43.model
--------------------
Mon Jul 23 15:53:13 2018: Reading training data ... 
Mon Jul 23 15:53:13 2018: Start ... #train=1405
--------------------
Forest-level: 
   loss=Log
   max_leaf

...


None
"predict": 
   model_fn=/tmp/rgf/e09d8a62-167f-44d4-88b8-29f8cb0d6ea91.model-99
   test_x_fn=/tmp/rgf/e09d8a62-167f-44d4-88b8-29f8cb0d6ea91.test.data.x
   prediction_fn=/tmp/rgf/e09d8a62-167f-44d4-88b8-29f8cb0d6ea91.predictions.txt
   Log:ON
--------------------
Mon Jul 23 15:55:09 2018: Reading test data ... 
Mon Jul 23 15:55:09 2018: Predicting ... 
elapsed: 0.029662
/tmp/rgf/e09d8a62-167f-44d4-88b8-29f8cb0d6ea91.predictions.txt: /tmp/rgf/e09d8a62-167f-44d4-88b8-29f8cb0d6ea91.model-99,#leaf=9900,#tree=1791
Mon Jul 23 15:55:09 2018: Done ... 

None
n_jobs = -1, but RGFClassifier uses one CPU because classes_ is 2
"train": 
   algorithm=RGF
   train_x_fn=/tmp/rgf/b7d22f2d-4365-42d5-8f21-182d18c79caa1.train.data.x
   train_y_fn=/tmp/rgf/b7d22f2d-4365-42d5-8f21-182d18c79caa1.train.data.y
   Log:ON
   model_fn_prefix=/tmp/rgf/b7d22f2d-4365-42d5-8f21-182d18c79caa1.model
--------------------
Mon Jul 23 15:54:47 2018: Reading training data ... 
Mon Jul 23 15:54:47 2018: Start ... #trai


None
"predict": 
   model_fn=/tmp/rgf/b7d22f2d-4365-42d5-8f21-182d18c79caa1.model-99
   test_x_fn=/tmp/rgf/b7d22f2d-4365-42d5-8f21-182d18c79caa1.test.data.x
   prediction_fn=/tmp/rgf/b7d22f2d-4365-42d5-8f21-182d18c79caa1.predictions.txt
   Log:ON
--------------------
Mon Jul 23 15:55:09 2018: Reading test data ... 
Mon Jul 23 15:55:09 2018: Predicting ... 
elapsed: 0.034026
/tmp/rgf/b7d22f2d-4365-42d5-8f21-182d18c79caa1.predictions.txt: /tmp/rgf/b7d22f2d-4365-42d5-8f21-182d18c79caa1.model-99,#leaf=9900,#tree=1844
Mon Jul 23 15:55:09 2018: Done ... 

None
n_jobs = -1, but RGFClassifier uses one CPU because classes_ is 2
"train": 
   algorithm=RGF
   train_x_fn=/tmp/rgf/0755546f-6024-493f-8362-97c4387c13412.train.data.x
   train_y_fn=/tmp/rgf/0755546f-6024-493f-8362-97c4387c13412.train.data.y
   Log:ON
   model_fn_prefix=/tmp/rgf/0755546f-6024-493f-8362-97c4387c13412.model
--------------------
Mon Jul 23 15:55:10 2018: Reading training data ... 
Mon Jul 23 15:55:10 2018: Start ... #trai


None
"train": 
   algorithm=RGF
   train_x_fn=/tmp/rgf/23ea7b74-6707-458c-b29c-b788294b160f2.train.data.x
   train_y_fn=/tmp/rgf/23ea7b74-6707-458c-b29c-b788294b160f2.train.data.y
   Log:ON
   model_fn_prefix=/tmp/rgf/23ea7b74-6707-458c-b29c-b788294b160f2.model
--------------------
Mon Jul 23 15:55:09 2018: Reading training data ... 
Mon Jul 23 15:55:09 2018: Start ... #train=1405
--------------------
Forest-level: 
   loss=Log
   max_leaf_forest=10000
   max_tree=5000
   opt_interval=100
   test_interval=100
   num_tree_search=1
   Verbose:ON
   memory_policy=Generous
-------------
Training data: 25x1405, nonzero_ratio=0.8521; managed as dense data.
-------------
Optimization: 
   loss=Log
   num_iteration_opt=5
   reg_L2=1
   opt_stepsize=0.5
   max_delta=1
Tree-level: min_pop=10
Node split: reg_L2=0.005
--------------------
Mon Jul 23 15:55:09 2018: Calling optimizer with 22 trees and 100 leaves
Mon Jul 23 15:55:09 2018: Writing model: seq#=1
Mon Jul 23 15:55:09 2018: Calling optim


None
"predict": 
   model_fn=/tmp/rgf/0755546f-6024-493f-8362-97c4387c13412.model-99
   test_x_fn=/tmp/rgf/0755546f-6024-493f-8362-97c4387c13412.test.data.x
   prediction_fn=/tmp/rgf/0755546f-6024-493f-8362-97c4387c13412.predictions.txt
   Log:ON
--------------------
Mon Jul 23 15:55:31 2018: Reading test data ... 
Mon Jul 23 15:55:31 2018: Predicting ... 
elapsed: 0.030335
/tmp/rgf/0755546f-6024-493f-8362-97c4387c13412.predictions.txt: /tmp/rgf/0755546f-6024-493f-8362-97c4387c13412.model-99,#leaf=9900,#tree=1753
Mon Jul 23 15:55:31 2018: Done ... 

None
n_jobs = -1, but RGFClassifier uses one CPU because classes_ is 2
"predict": 
   model_fn=/tmp/rgf/23ea7b74-6707-458c-b29c-b788294b160f2.model-99
   test_x_fn=/tmp/rgf/23ea7b74-6707-458c-b29c-b788294b160f2.test.data.x
   prediction_fn=/tmp/rgf/23ea7b74-6707-458c-b29c-b788294b160f2.predictions.txt
   Log:ON
--------------------
Mon Jul 23 15:55:31 2018: Reading test data ... 
Mon Jul 23 15:55:31 2018: Predicting ... 
elapsed: 0.030942



None
"predict": 
   model_fn=/tmp/rgf/e21d85e0-2e6a-46af-9a1d-48eaf8d4a9363.model-99
   test_x_fn=/tmp/rgf/e21d85e0-2e6a-46af-9a1d-48eaf8d4a9363.test.data.x
   prediction_fn=/tmp/rgf/e21d85e0-2e6a-46af-9a1d-48eaf8d4a9363.predictions.txt
   Log:ON
--------------------
Mon Jul 23 15:55:54 2018: Reading test data ... 
Mon Jul 23 15:55:54 2018: Predicting ... 
elapsed: 0.030759
/tmp/rgf/e21d85e0-2e6a-46af-9a1d-48eaf8d4a9363.predictions.txt: /tmp/rgf/e21d85e0-2e6a-46af-9a1d-48eaf8d4a9363.model-99,#leaf=9900,#tree=1871
Mon Jul 23 15:55:54 2018: Done ... 

None


In [18]:
print('RGF precision: %.3f +/- %.3f' % (np.mean(pres),np.std(pres)))
print('RGF recall: %.3f +/- %.3f' % (np.mean(recall),np.std(recall)))
print('RGF f1: %.3f +/- %.3f' % (np.mean(f1),np.std(f1)))

RGF precision: 0.934 +/- 0.047
RGF recall: 0.906 +/- 0.068
RGF f1: 0.919 +/- 0.050


In [15]:
from sklearn.tree import DecisionTreeClassifier
DT=DecisionTreeClassifier()
scores=cross_val_score(estimator=DT,scoring='precision',X=X_train,y=y_train,cv=5,n_jobs=2)
print('DT replaced precision: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))
scores=cross_val_score(estimator=DT,scoring='recall',X=X_train,y=y_train,cv=5,n_jobs=2)
print('DT replaced recall: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))
scores=cross_val_score(estimator=DT,scoring='f1',X=X_train,y=y_train,cv=5,n_jobs=2)
print('DT replaced f1: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

DT replaced precision: 0.875 +/- 0.050
DT replaced recall: 0.891 +/- 0.041
DT replaced f1: 0.883 +/- 0.045
